In [1]:
#!/bin/bash
""" " 2>/dev/null 
# Run python in inf loop
while true; do python3 $0 "${@}"; sleep 0.1; done
" """
COUNTDOWN = 10
DO_DELETE = False
DO_MD5_CHECK = False
HD_LARGER_THAN_THIS_GB = 400

#requirements psutils termcolor tqdm
#apt-get install ntfs-3g gnome-terminal

import drive_utils
import termwriter 
import filelogger
import misc
import copier
from termwriter import *

import os
#from termcolor import colored as col
from tqdm import tqdm
import shutil
import time
from datetime import datetime

if misc.jupyter_mode():
    tqdm.monitor_interval = 0
    get_ipython().run_line_magic('reload_ext', 'autoreload')
    get_ipython().run_line_magic('autoreload', '2')
    misc.save_jupyter_as_bash('media-copier.ipynb')

In [ ]:
vid_ext, img_ext = misc.get_video_image_ext()
poll = drive_utils.drivePollster()

status = misc.dotdict(
    terminated = False,
    uncopied = 0)


ii=-1
while(1):
    ii+=1
    
    #--------------------------------------------------------
    #Get all the drive logic ready
    drives = drive_utils.get_drives()

    #Use largest USB drive as harddrive
    hdd = None
    for drive in drives:
        if drive_utils.is_usb_drive(drive.device):
            if drive.size_total > HD_LARGER_THAN_THIS_GB*(2**30):
                hdd = drive

    #Use smaller USB drive as SSD
    ssd_list = []
    for drive in drives:
        if drive_utils.is_usb_drive(drive.device):
            if drive.size_total <= HD_LARGER_THAN_THIS_GB*(2**30):
                ssd_list.append(drive)
                #lets use this for some additional info
                ssd_list[-1].inf = misc.dotdict()
    
    #--------------------------------------------------------
    #Triggers: when devices are plugged in/out
    if poll.did_change() or ii==0: 
        status.terminated = False
        status.uncopied = 0
        start = time.time()
    
    #--------------------------------------------------------
    # Crude color display
    c = coldict
    #HEADER OKBLUE OKGREEN WARNING FAIL ENDC BOLD UNDERLINE
    
    termwriter.cls()
    cols, lines = shutil.get_terminal_size((80, 20))
    if not (hdd and len(ssd_list)):
        printterm(c.RED, end='')
        writeterm("-"*cols, '+', '+')
        writeterm('', '|', '|')
        writeterm(c.RED+' Please insert' +c.BLUE+' USB Hard drive' +c.RED+' and' +c.GREEN+' Camera SD' +c.RED
                        +'.'*(ii%4+1), '|','|')
        writeterm('', '|', '|')
        writeterm("-"*cols, '+', '+')
        printterm(c.END, end='')

    else:
        #--------------------------------------------------------
        # Copy from SSD to hard-drive
        printterm(c.BLUE, end='')
        writeterm("----- Copy From "+'-'*cols, '+', '+')
        for ssd in ssd_list:
            writeterm('   '+ssd.mountpoint, '|','|')
            writeterm("     used: %.1f GB"%(ssd.size_used/(2**30)), '|','|')
        #writeterm("-"*cols, '+', '+')
        printterm(c.GREEN, end='')
        writeterm("----- Copy To "+'-'*cols, '+', '+')
        writeterm('   '+hdd.mountpoint, '|','|')
        writeterm("     free:  %.1f GB"%(hdd.size_free/(2**30)), '|','|')
        writeterm("     (tot:  %.1f GB)"%(hdd.size_total/(2**30)), '|','|')
        writeterm("-"*cols, '+', '+')
        printterm(c.END, end='')
        
        #Do not copy if the routine is already finished
        if status.terminated:
            printterm()
            if status.uncopied > 0:
                printterm("SD card %.3fGB uncopied! Drive too full!"%(status.uncopied/2**30))
                printterm("Replace harddrive...")
            else:
                printterm("SD card backup is complete!")
                printterm("You may remove devices...")
            continue
        
        #--------------------------------------------------------
        # Wait 20 second
        printterm()
        if (time.time()-start) < COUNTDOWN:
            printterm(' %d s left to cancel (plug out device)...'%(
                COUNTDOWN - (time.time()-start)))
            continue
            
        #--------------------------------------------------------
        # Do the copy
        printterm(" Scanning files:")

        hdd_dir = os.path.join(hdd.mountpoint, 'SD_autobackup')


        os.makedirs(hdd_dir, exist_ok=True)

        fileregister = filelogger.get_fileregister(hdd_dir)
        newregister = {}


        t = tqdm(total=sum(i.size_used for i in ssd_list),
                 file=sysout, unit='B', unit_scale=True)
        t_progress = 0
        totsize = 0
        filecount = misc.dotdict(files=0, size=0, vidfiles=0, vidsize=0,
                                                  imgfiles=0, imgsize=0)
        for ssd in ssd_list:
            #force full path with trailing /
            ssd.inf.abspath = os.path.abspath(os.path.join(ssd.mountpoint,''))
            for _, dirs, files in os.walk(ssd.inf.abspath, followlinks=False):
                for file in files:
                    fpath = os.path.join(_, file)
                    try:
                        key, attrs = filelogger.to_register(fpath)
                    except FileNotFoundError:
                        pass
                        #print('File not found ', fpath)
                    else:
                        if key not in fileregister:
                            newregister[key] = attrs

                            filecount.files += 1
                            filecount.size  += attrs.size

                            if os.path.splitext(attrs.path)[-1] in vid_ext:
                                filecount.vidfiles += 1
                                filecount.vidsize  += attrs.size

                            if os.path.splitext(attrs.path)[-1] in img_ext:
                                filecount.imgfiles += 1
                                filecount.imgsize  += attrs.size

                    t_progress += attrs.size
                    t.update(attrs.size)
            t.update(sum(i.size_used for i in ssd_list) - t_progress) #if not 100% yet
        t.close()
        printterm('\r', end='')

        available_space = hdd.size_free*0.9999 #leave 100kB tolerance / GB
        
        printterm('Uncopied videos %.2fGB; %d files.'%(filecount.vidsize/2**30, filecount.vidfiles))
        printterm('Uncopied images %.2fGB; %d files.'%(filecount.imgsize/2**30, filecount.imgfiles))
        printterm('Uncopied all files %.2fGB; %d files.'%(filecount.size/2**30, filecount.files))
        
        #Determine max space that can be occupied
        if filecount.size > available_space:
            size_fill = 0
            size_left = 0
            for key, attrs in newregister.items():
                if size_fill + attrs.size < available_space:
                    size_fill += attrs.size
                else:
                    size_left += attrs.size
            
            printterm('Warning!!! Full drive; %.3fGB files abandoned.'%(
                (size_left)/2**30)+'%')
            
        printterm()

        for ssd in ssd_list:
            ssd.inf.out_dir = os.path.join(hdd_dir,
                                    datetime.now().strftime('%Y%m%d-')+
                                    copier.get_valid_filename(ssd.id)+'--%.1fGB'%(
                                       ssd.size_total/(2**30)))
        
        i=0
        t = tqdm(total=filecount.size, file=sysout, unit='B', unit_scale=True)
        for key, attrs in newregister.items():
            i+=1

            src = attrs.path
                
            available_space -= attrs.size
            #once triggered, always triggered :)
            if available_space<0: 
                status.uncopied += attrs.size
                continue

            #From which sd card is this file???
            for s in ssd_list:
                if os.path.abspath(src).startswith(ssd.inf.abspath):
                    ssd = s
                    break

            # +'/'+ is alternative to path.join; otherwise it breaks on filenames starting with '.'
            dest = os.path.abspath(ssd.inf.out_dir +'/'+ src[len(ssd.inf.abspath):])
            dest_dir = os.path.dirname(dest)


            if not os.path.isdir(dest_dir):
                os.makedirs(dest_dir)

            keeps = misc.dotdict()
            for chunk_sz in copier.copy_with_progress(src, dest,
                                                     do_md5=DO_MD5_CHECK,
                                                     keeps=keeps):
                t.update(chunk_sz)

            safe = (copier.hash_equals(keeps.md5, copier.hash_file(dest)) if DO_MD5_CHECK else 
                    os.stat(src).st_size == os.stat(dest).st_size)

            if safe:
                filelogger.update_fileregister(hdd_dir, attrs)
                if DO_DELETE:
                    try:
                        os.remove(src)
                    except: pass

        t.close()
        printterm('\r', end='')
        
        for ssd in ssd_list:
            misc.unmount(ssd)
            
        status.terminated = True

    time.sleep(0.5)